In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

For this lecture you will want to have installed additional Python packages:


In [2]:
!pip install line-profiler numexpr cython


# High Performance Python

Python is slower than compiled languages for a variety of reasons:

### Python is Dynamically Typed rather than Statically Typed.

What this means is that at the time the program executes, the interpreter doesn't know the type of the variables that are defined. For example, the difference between a C variable and a Python variable is summarized by this diagram:

![](http://jakevdp.github.io/images/cint_vs_pyint.png)

For a variable in C, the compiler knows the type by its very definition. For a variable in Python, all you know at the time the program executes is that it's some sort of Python object.

So if you write the following in C:

```C
int a = 1;
int b = 2;
int c = a + b;
```

the C compiler knows from the start that a and b are integers: they simply can't be anything else! With this knowledge, it can call the routine which adds two integers, returning another integer which is just a simple value in memory. As a rough schematic, the sequence of events looks like this:

**C Addition**

1. Assign <int> 1 to a
2. Assign <int> 2 to b
3. call binary_add<int, int>(a, b)
4. Assign the result to c

The equivalent code in Python looks like this:

```python
a = 1
b = 2
c = a + b
```

here the interpreter knows only that 1 and 2 are objects, but not what type of object they are. So the The interpreter must inspect PyObject_HEAD for each variable to find the type information, and then call the appropriate summation routine for the two types. Finally it must create and initialize a new Python object to hold the return value. The sequence of events looks roughly like this:

**Python Addition**

1. Assign 1 to a
    - Set a->PyObject_HEAD->typecode to integer
    - Set a->val = 1
2. Assign 2 to b
    - Set b->PyObject_HEAD->typecode to integer
    - Set b->val = 2
3. call binary_add(a, b)
    - find typecode in a->PyObject_HEAD
    - a is an integer; value is a->val
    - find typecode in b->PyObject_HEAD
    - b is an integer; value is b->val
    - call binary_add<int, int>(a->val, b->val)
    - result of this is result, and is an integer.
4. Create a Python object c
    - set c->PyObject_HEAD->typecode to integer
    - set c->val to result

The dynamic typing means that there are a lot more steps involved with any operation. This is a primary reason that Python is slow compared to C for operations on numerical data.

### Python is interpreted rather than compiled.

We saw above one difference between interpreted and compiled code. A smart compiler can look ahead and optimize for repeated or unneeded operations, which can result in speed-ups. Compiler optimization is its own beast, and I'm personally not qualified to say much about it, so I'll stop there. For some examples of this in action, you can take a look at my previous post on Numba and Cython.

### Python's object model can lead to inefficient memory access

We saw above the extra type info layer when moving from a C integer to a Python integer. Now imagine you have many such integers and want to do some sort of batch operation on them. In Python you might use the standard List object, while in C you would likely use some sort of buffer-based array.

A NumPy array in its simplest form is a Python object build around a C array. That is, it has a pointer to a contiguous data buffer of values. A Python list, on the other hand, has a pointer to a contiguous buffer of pointers, each of which points to a Python object which in turn has references to its data (in this case, integers). This is a schematic of what the two might look like:

![](http://jakevdp.github.io/images/array_vs_list.png)

It's easy to see that if you're doing some operation which steps through data in sequence, the numpy layout will be much more efficient than the Python layout, both in the cost of storage and the cost of access.

## Speeding up statistical computations in Python using `numexpr` and `cython`

In the age of "big data" and sophisitcated Bayesian and statistical learning algorithms, many are interested in optimizing the performance of the high-level languages that we use to analyse data.

[NumPy](http://numpy.scipy.org/) gets us part of the way there on Python:

* Storage of multidimensional data
* Efficient data access
* Efficient in-memory storage
* Fast methods and functions for data manipulation

This gets us part of the way there, and for many applications it is sufficient. However, there is plenty of scope for improving Python's performance in situations where speed matters.

Pure Python and Python with NumPy are not particularly fast. Below are some recent performance benchmarks comparing several computing languages (taken directly from the [Julia website](http://julialang.org)):


<div class="figure">
<table class="benchmarks">
<colgroup>
<col class="name"></col>
<col class="relative" span="9"></col>
</colgroup>
<thead>
<tr style="background-color:#EEEEEE;"><td></td><th class="system">Fortran</th><th class="system">Julia</th><th class="system">Python</th><th class="system">R</th><th class="system">Matlab</th><th class="system">Octave</th><th class="system">Mathematica</th><th class="system">JavaScript</th><th class="system">Go</th></tr>
<tr style="background-color:#EEEEEE;"><td></td><td class="version">gcc 4.8.1
</td><td class="version">0.2</td><td class="version">2.7.3
</td><td class="version">3.0.2
</td><td class="version">R2012a
</td><td class="version">3.6.4
</td><td class="version">8.0
</td><td class="version">V8 3.7.12.22
</td><td class="version">go1
</td></tr>
</thead>
<tbody>
<tr><th>fib</th><td class="data">0.26</td><td class="data">0.91</td><td class="data">30.37</td><td class="data">411.36</td><td class="data">1992.00</td><td class="data">3211.81</td><td class="data">64.46</td><td class="data">2.18</td><td class="data">1.03</td></tr>
<tr><th>parse_int</th><td class="data">5.03</td><td class="data">1.60</td><td class="data">13.95</td><td class="data">59.40</td><td class="data">1463.16</td><td class="data">7109.85</td><td class="data">29.54</td><td class="data">2.43</td><td class="data">4.79</td></tr>
<tr><th>quicksort</th><td class="data">1.11</td><td class="data">1.14</td><td class="data">31.98</td><td class="data">524.29</td><td class="data">101.84</td><td class="data">1132.04</td><td class="data">35.74</td><td class="data">3.51</td><td class="data">1.25</td></tr>
<tr><th>mandel</th><td class="data">0.86</td><td class="data">0.85</td><td class="data">14.19</td><td class="data">106.97</td><td class="data">64.58</td><td class="data">316.95</td><td class="data">6.07</td><td class="data">3.49</td><td class="data">2.36</td></tr>
<tr><th>pi_sum</th><td class="data">0.80</td><td class="data">1.00</td><td class="data">16.33</td><td class="data">15.42</td><td class="data">1.29</td><td class="data">237.41</td><td class="data">1.32</td><td class="data">0.84</td><td class="data">1.41</td></tr>
<tr><th>rand_mat_stat</th><td class="data">0.64</td><td class="data">1.66</td><td class="data">13.52</td><td class="data">10.84</td><td class="data">6.61</td><td class="data">14.98</td><td class="data">4.52</td><td class="data">3.28</td><td class="data">8.12</td></tr>
<tr><th>rand_mat_mul</th><td class="data">0.96</td><td class="data">1.01</td><td class="data">3.41</td><td class="data">3.98</td><td class="data">1.10</td><td class="data">3.41</td><td class="data">1.16</td><td class="data">14.60</td><td class="data">8.51</td></tr>
</tbody>
</table>

<p class="caption"><b>Figure:</b>
benchmark times relative to C (smaller is better, C performance = 1.0).
</p>
</div>

So, while fast relative to some scientific compution choices (*e.g.* R, Matlab), Python sometimes needs to be tweaked in order to make it a competitive choice for implementing modern statistical methods. We will cover two approachable ways of improving the performance of Python.


## Profiling

Before you barrel ahead and prematurely optimize your Python code, it is important to understand **why** and **where** your code is slow. This is achieved by systematically accounting for the resources that your code is using, such as memory, CPU time or data transfer. This process is broadly referred to as ***Profiling***, and it allows you to identify where the performance bottlenecks in your code lie.

Here, we will concentrate on optimizing performance for **CPU-bound** problems.

There are a number of tools to help you profile your code.

### `time`

For those of you on UNIX platforms, the built-in utility `time` can be used to assess how long your code takes to run.

In [3]:
!time python ../examples/abc.py

mu       4.203959
sigma    2.007131
dtype: float64

real	0m2.589s
user	0m2.238s
sys	0m0.206s


The output from `time` can be interpreted as:

* `real`: elapsed (wall) time
* `user`: time spent in your code
* `sys`: time spent in system (kernel) functions

The last 2 quantities account for the cycles used by your program. The remaining `real` time is often due to waiting for information either from disk or a network connection (I/O).

Python also has a `time` module (and function) that is more rudimentary; it simply returns the time, in seconds from the Epoch (1/1/1970).

In [4]:
import time
time.time()

1449149480.620591

We can use this for profiling by differencing the times before and after running some code of interest:

In [5]:
import numpy as np
start_time = time.time()
np.product(range(1, 100000))
end_time = time.time()

end_time - start_time

0.02634286880493164

Note, however that it does not provide a breakdown of where the code spends its time.

### IPython magic: `%timeit`, `%run` and `%prun`

IPython has three built-in "magic" functions that are useful for profiling your code. 

The `%timeit` magic executes a Python statement or expressions in a loop to see how long we expect it to take for any given call. Additionally, it repeats the loop a certain number of times, and returns the best result.

As an example, consider a Python implementation of the **trapezoidal rule**, a method from numerical analysis for approximating a definite integral. Specifically, it allows us to approximate:

$$\int_a^b f(x) dx$$

using the approximation:

$$\int_a^b f(x) dx \approx (b-a) \frac{f(b) + f(a)}{2}$$

Rather than use a single interval for this estimate, we break the interval down into $n$ subintervals, to obtain a more accurate approximation.

In [6]:
def f(x):
    return 2*x*x + 3*x + 1
      
def trapez(a, b, n):
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    
    for i in range(n):
        x += h
        sumy += f(x)
    sumy += 0.5*(f(a) + f(b))
    return sumy*h

In [7]:
trapez(1, 5, 10000)

122.6930668799846

To confirm that this works, we can compare this to the symbolic solution, using Sympy:

In [8]:
from sympy import init_printing
init_printing()
import sympy as sym

In [9]:
xs = sym.symbols('xs')

fx = 2*xs*xs + 3*xs + 1

ifx = sym.integrate(fx, (xs, 1, 5))
ifx.evalf()

In [10]:
%timeit trapez(1, 5, 10000)

100 loops, best of 3: 5.2 ms per loop


`%timeit` tries to pick suitable values for the number of loops and repeats; these values can be overriden by specifying `-n` and `-r` values, respectively.

The `%run` command with a `-p` option allows you to run complete programs under the control of the Python profiler. It writes the output to the help pane, which opens at the bottom of the page.

In [11]:
# This code redirects pager output to a regular cell
from IPython.core import page
page.page = print

In [12]:
%run -p ../examples/abc.py

mu       4.335902
sigma    2.283576
dtype: float64
         1414863 function calls (1412264 primitive calls) in 2.396 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    40694    0.593    0.000    1.094    0.000 _methods.py:76(_var)
   124114    0.393    0.000    0.393    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    40694    0.161    0.000    0.454    0.000 _methods.py:53(_mean)
    81388    0.158    0.000    0.172    0.000 _methods.py:43(_count_reduce_items)
    40694    0.150    0.000    1.262    0.000 _methods.py:122(_std)
    40695    0.134    0.000    0.134    0.000 {method 'normal' of 'mtrand.RandomState' objects}
   124106    0.112    0.000    0.138    0.000 numeric.py:476(asanyarray)
        1    0.106    0.106    2.036    2.036 abc.py:4(abc)
      155    0.089    0.001    0.089    0.001 {built-in method marshal.loads}
   248935    0.052    0.000    0.052    0.000 {built-in method builtins.isinstance}
   125

The profiling information includes the following information:

* `ncalls`: number of calls to function
* `tottime`: total time spent in the given function (excluding time in calls to sub-functions)
* `percall`: time per call
* `cumtime`: cumulative time spent in this and all subfunctions 

We can see that most of the time in this example is spent inside of core NumPy functions and methods.

The `%prun` command does a similar job for single Python expressions (like function calls).

In [13]:
%prun trapez(2, 6, 100000)

         100006 function calls in 0.076 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100002    0.040    0.000    0.040    0.000 <ipython-input-6-262718d3c827>:1(f)
        1    0.036    0.036    0.076    0.076 <ipython-input-6-262718d3c827>:4(trapez)
        1    0.000    0.000    0.076    0.076 {built-in method builtins.exec}
        1    0.000    0.000    0.076    0.076 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
 

For even more fine-grained profiling information, we can use the line profiler (`line-profiler` package) to see how long it takes each line of a function to run.

In [14]:
%load_ext line_profiler

In [15]:
run -i ../examples/bisection.py

In [16]:
%lprun -f bisection bisection(lambda x: -2*x + 5, -10, 10)

Timer unit: 1e-06 s

Total time: 0.000215 s
File: /Users/fonnescj/Github/Bios8366/examples/bisection.py
Function: bisection at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def bisection(f, a, b, tol=1e-9, max_iter=100):
     2                                           
     3                                               # Check initial values
     4         1            5      5.0      2.3      if a >= b:
     5                                                   raise ValueError('Right value must be larger than left')
     6                                           
     7         1            3      3.0      1.4      fa, fb = f(a), f(b)
     8                                           
     9         1            1      1.0      0.5      if fa*fb > 0:
    10                                                   raise ValueError('No maximum between specified values')
    11                                          

This output makes it clear that the biggest cost is in the repeated calling of the function $f$ for which the root is being found. If we could improve the speed of this function, it would be the easiest single way of improving the performance of the code.


## Fast array expression evaluation with `numexpr`

[`numexpr`](http://code.google.com/p/numexpr/) allows array expressions to be evaluated far faster that what can be achieved in Python using Numpy arrays. `numexpr` parses a string expression and optimizes and compiles the code on the fly, using a virtual machine that includes a [Just-in-time (JIT) compiler](http://en.wikipedia.org/wiki/Just-in-time_compilation). In addition, `numexpr` offers direct support for parallel multi-threaded computations, since Python's global interpreter lock is bypassed.

> Python's global interpreter lock (GIL) ensures that only one thread runs in the interpreter at once. This simplifies many of the low-level activities, such as memory management, and allows for co-operative multi-tasking. But, since the currently-running thread holds onto the interpreter, it makes multi-core parallelization difficult.

Part the reason Python can be slow for array calculations is that it creates temporary arrays to store intermediate results from array element calculations, which wastes memory and cache. `numexpr` handles such calculations in manageable chunks, which accellerates computation.

The speedup over NumPy by using `numexpr` can be as high as 20x, but is typically in the range of 2-4x.

### Example: Computing a polynomial

In [17]:
import numpy as np

x = np.linspace(-1, 1, 1e7)

0.25*x**3 + 0.75*x**2 - 1.5*x - 2

array([  0.00000000e+00,  -4.50000045e-07,  -9.00000090e-07, ...,
        -2.50000030e+00,  -2.50000015e+00,  -2.50000000e+00])

In [18]:
%timeit 0.25*x**3 + 0.75*x**2 - 1.5*x - 2

1 loops, best of 3: 613 ms per loop


In [19]:
import numexpr as ne

ne.set_num_threads(1)

In [20]:
ne.evaluate('0.25*x**3 + 0.75*x**2 - 1.5*x - 2')

array([  0.00000000e+00,  -4.50000045e-07,  -9.00000090e-07, ...,
        -2.50000030e+00,  -2.50000015e+00,  -2.50000000e+00])

In [21]:
%timeit ne.evaluate('0.25*x**3 + 0.75*x**2 - 1.5*x - 2')

10 loops, best of 3: 89.5 ms per loop


`numexpr` actually expands the polynomial terms so that it does not need to use a transcendental function.

We can achieve further gains in performance by multithreading the calculations:

In [22]:
ne.set_num_threads(4)

In [23]:
%timeit ne.evaluate('0.25*x**3 + 0.75*x**2 - 1.5*x - 2')

10 loops, best of 3: 40.4 ms per loop


Since the performance of processors has outpaced that of memory in the past several decades, the CPU spends a lot of time waiting for memory to give it computations; this is the ***processor-memory performance gap***.

![performance gap](http://www.techdesignforums.com/practice/files/2013/02/tdf-snps-ARMcc-feb13-fig1lg.jpg)
(graph courtesy http://www.techdesignforums.com)

CPU caches are often used to make up for this difference. CPU caches are more effective when the data are optimally located in memory to take advantage of cache performance. `numexpr` does this by moving contiguous blocks of data from memory to the CPU cache, reusing them as much as possible within the cache to more quickly give the CPU access to the data.

### Limitations

`numexpr` only implements element-wise operations. So, `a * b` becomes:

    for i in range(N):
        c[i] = a[i] * b[i]

Similarly, it cannot index other parts of arrays in the same expression:

    for i in range(N):
        c[i] = a[i-1] + a[i] * b[i]

## Cython

Python developers typically solve performance constraints by building Python extensions by wrapping code written in other languages (for example, SciPy contains more lines of C/C++/Fortran than Python). However, programming with the Python/C API is not straightforward for most users.

Cython is a language that allows Python programmers to write fast code without having to write C/C++/Fortran directly. It looks much like Python code, but with type declarations. Cython code is translated it to C (or C++ or others), which is then compiled to create a Python extension that we can import and use. 

Using Cython, we can achieve speedups of several orders of magnitude, often *faster than hand-coded C code*. In addtion, Cython is compatible with core scientific programming tools like NumPy and IPython.

Cython has built-in support for multicore processing.

Cython is used to varying degrees by other packages in the Python scientific stack, such as pandas, sympy, scikit-learn and SciPy.

### Example: Numerical integration

Recall from above the function `trapez` for performing numerical integration using the trapezoidal rule. As a benchmark, let's time the execution of the pure-Python version of the trapezoidal rule:

In [24]:
%timeit trapez(1, 5, 1000)

1000 loops, best of 3: 493 µs per loop


Perhaps the easiest way to use Cython, is via the IPython `cythonmagic`, which allows us to run Cython interactively:

In [25]:
%load_ext cython

In [26]:
%%cython

def f(x):
    return 2*x*x + 3*x + 1

def trapez2(a, b, n):
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    for i in range(n):
        x += h
        sumy += f(x)
    sumy += 0.5*(f(a) + f(b))
    return sumy*h

The Cython magic is doing a lot of work for you: it compiles the code into an extension module, and loads it into the notebook. This allows us to ignore all of the compilation details of building Cython extensions. 

If we run `trapez2`, we can see a reasonable speedup simply by compiling it, unchanged, using Cython.

In [32]:
%timeit trapez2(1, 5, 1000)

1000 loops, best of 3: 243 µs per loop


Under the hood, several things are happening in order to deliver this improved performance. The Cython source code is translated into C source code by `cython`. Then, this C source is compiled, using the appropriate compiler, flags and associated library files (if any), into a Python extension. This extension is then loaded by IPython into the current session.

![cython flow](images/cython.png)

C extensions can also be compiled manually, using a setup file. Here is an example for an extension called `dist` within a package called `probability`:

```python
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext

import numpy as np

setup(
    cmdclass = {'build_ext': build_ext},
    ext_modules = [Extension("dist", ["probability/src/dist.pyx"], include_dirs=[np.get_include()])]
)
```
    
It mainly uses machinery from a core Python package `distutils` that manages the build process.

If we look at the `trapez2` function compared to the pure Python `trapez`, we see that the Cython version appears to be a lower-level function:

In [27]:
trapez?

Signature: trapez(a, b, n)
Docstring: <no docstring>
File:      ~/Github/Bios8366/notebooks/<ipython-input-6-262718d3c827>
Type:      function


In [28]:
trapez2?

Docstring: <no docstring>
Type:      builtin_function_or_method


To get a closer look at where Cython is improving our unchanged Python code, we can add an `--annotate` flag to the `%%cython` magic declaration:

In [29]:
%%cython --annotate

def f(x):
    return 2*x*x + 3*x + 1
      
def trapez2(a, b, n):
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    for i in range(n):
        x += h
        sumy += f(x)
    sumy += 0.5*(f(a) + f(b))
    return sumy*h

In the above, the line color indicates the "typedness" of the extension, where yellower lines are closer to Python, and therefore require calls to the Python C API, while whiter lines indicate code that is closer to pure C, hence requiring few, if any, Python API calls.

If you click on a line, it unravels to show you the C code that results from the call to `cython`.

The goal in speeding up code with Cython is to turn as many lines to white as we can. The easiest way to do this is to add type declarations to the Python code:

In [30]:
%%cython --annotate

# Add type to argument
def ff(double x):
    return 2*x*x + 3*x + 1

# Add types to arguments
def trapez3(double a, double b, int n):
    # Declare types of variables
    cdef double h, x, sumy
    cdef int i
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    for i in range(n):
        x += h
        sumy += ff(x)
    sumy += 0.5*(ff(a) + ff(b))
    return sumy*h

In [31]:
%timeit trapez3(1, 5, 1000)

10000 loops, best of 3: 65.3 µs per loop


This gives us a considerable speedup. The next thing we might try is to *inline* the polynomial function. By inlining, we mean that we ask the compiler to perform an inline expansion of said function; that is, it will insert a copy of the function itself wherever the function is called, instead of calling the function wherever it is defined.

We do three things to the specification of `ff`:

* change `def` to `cdef`
* add a return type to the function
* add an `inline` keyword

In [33]:
%%cython --annotate

cdef inline double ff(double x):
    return 2*x*x + 3*x + 1

cpdef trapez4(double a, double b, int n):
    cdef double h, x, sumy
    cdef int i
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    for i in range(n):
        x += h
        sumy += ff(x)
    sumy += 0.5*(ff(a) + ff(b))
    return sumy*h

The `cdef` keyword declares a C object. Everything that follows it is therefore specified in terms of C; we are essentially writing C, but using a subset of Python's syntax rules. So, when we are creating a function `cdef ff` it is a C function, and is not available to you in Python.

`cpdef` is a hybrid declaration that creates both a C interface and a Python interface to the function.

Let's see how this performs.

In [34]:
%timeit trapez4(1, 5, 1000)

100000 loops, best of 3: 2.74 µs per loop


Woof! That's a big speedup, and there's not much yellow left in the annotated code. 

If you would like a very simple way of injecting types into your code with Cython, without modifying any of the code itelf, you can use the `@cython.locals` decorator. Note that you don't get as fast of a speedup as we have just achieved.

In [35]:
%%cython
import cython

@cython.locals(x=cython.double)
def f(x):
    return 2*x*x + 3*x + 1
     
@cython.locals(a=cython.double, b=cython.double, n=cython.int,
               h=cython.double, sumy=cython.double, i=cython.int,
               x=cython.double, func=cython.double)
def trapez5(a, b, n):
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    
    for i in range(n):
        x += h
        sumy += f(x)
    sumy += 0.5*(f(a) + f(b))
    return sumy*h

In [36]:
%timeit trapez5(1, 5, 1000)

10000 loops, best of 3: 63.1 µs per loop


If you can stand to look at it, you can peek at all the C code that is generated by Cython just to optimize this short function.

In [ ]:
%load ../examples/trapezoid.c

Due to conveneince, running Cython from IPython is a preferred way of using the language. However, if we have some **legacy C/C++ code** that we wish to use in Python, we can do that by writing a wrapper and calling `cython` from the terminal.

Here is the C code:

In [ ]:
%load ../examples/fact.h

And here is the Cython wrapper. Cython code is stored in files with a `.pyx` extension.

In [ ]:
%load ../examples/fact.c

In [ ]:
%load ../examples/fact.pyx

In [37]:
! cython ../examples/fact.pyx

Now we can compile the extension.

In [39]:
import os
os.chdir('../examples')

!gcc -Wall -fno-strict-aliasing -static -undefined dynamic_lookup \
    -bundle -arch x86_64 \
    -I/Users/fonnescj/anaconda3/include/python3.5m \
    -o fact.so fact.c
    
os.chdir('../notebooks')

fact.c:1506:28: warning: unused function '__Pyx_PyObject_AsString' [-Wunused-function]
static CYTHON_INLINE char* __Pyx_PyObject_AsString(PyObject* o) {
                           ^
fact.c:1503:32: warning: unused function '__Pyx_PyUnicode_FromString' [-Wunused-function]
static CYTHON_INLINE PyObject* __Pyx_PyUnicode_FromString(const char* c_str) {
                               ^
fact.c:1568:26: warning: unused function '__Pyx_PyObject_IsTrue' [-Wunused-function]
static CYTHON_INLINE int __Pyx_PyObject_IsTrue(PyObject* x) {
                         ^
fact.c:1618:33: warning: unused function '__Pyx_PyIndex_AsSsize_t' [-Wunused-function]
static CYTHON_INLINE Py_ssize_t __Pyx_PyIndex_AsSsize_t(PyObject* b) {
                                ^
fact.c:1680:33: warning: unused function '__Pyx_PyInt_FromSize_t' [-Wunused-function]
static CYTHON_INLINE PyObject * __Pyx_PyInt_FromSize_t(size_t ival) {
                                ^
fact.c:1248:32: warning: unused function '__Pyx_PyInt_From_l

In [40]:
!cp ../examples/fact.so .

In [41]:
import fact

fact.fact(5)

### Using lists and arrays in Cython

The above example used only scalar variables. When we have vector-valued data, we need to declare the appropriate types. Here's a simple example, using a function that calculates the Euclidean distance between two arrays:

In [42]:
def euclidean(x, y):
    x = np.array(x)
    y = np.array(y)
    return np.sqrt(((x - y) ** 2).sum())

In [43]:
%timeit euclidean(np.random.randn(10), np.random.randn(10))

The slowest run took 18.67 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 10.6 µs per loop


In order to get a speedup under Cython, we need to iterate over the elements of each passed array, and aggregate them manually.

In [44]:
%%cython --annotate

import cython
cimport numpy as np
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def euclidean2(np.ndarray[np.float64_t, ndim=1] x, 
               np.ndarray[np.float64_t, ndim=1] y):
    cdef: 
        double diff
        int i
    diff = 0
    for i in range(x.shape[0]):
        diff += (x[i] - y[i])**2
    return sqrt(diff)


In [45]:
%timeit euclidean2(np.random.randn(10), np.random.randn(10))

The slowest run took 15.49 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 5.48 µs per loop


The decorators for `trapez5` are **compiler directives** that alter the behavior of Cython code. Setting `boundscheck` to False removes boundary checking for indexing operations, forcing us to ensure that we do not try to index arrays using index vlaues that are out of bounds. When we set `wraparound` to False, Cython will not support negative indexes, as is the case with Python. While these directives may increase the speed of our code, it can be dangerous; if we do not ensure that we index our arrays properly, it may cause segmentation faults or data corruption.

The full set of compiler directives are described in the [Cython docs](http://docs.cython.org/src/reference/compilation.html#compiler-directives).

Here is the same code using lists instead of NumPy arrays:

In [46]:
%%cython --annotate

from libc.math cimport sqrt

def euclidean3(list x, list y):
    cdef: 
        double diff
        int i
    diff = 0
    for i in range(len(x)):
        diff += (x[i] - y[i])**2
    return sqrt(diff)


In [47]:
%timeit euclidean3(np.random.randn(10).tolist(), np.random.randn(10).tolist())

The slowest run took 13.46 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 6.01 µs per loop


### `pyximport`

If we have some Cython source code, we can use `pyximport` to directly import it as if it were a Python module.

In [48]:
import pyximport

# Move source into current directory
!cp ../examples/trapezoid.pyx .

# Allow it to use Python's import mechanism
pyximport.install()

from trapezoid import trapez as trapez_pyx
trapez_pyx(1, 10, 10) 

In other words, it treats `.pyx` files as if they were `.py` files. This includes detecting changes in the source file,, if any, and recompiling it as necessary before importing.

### Benchmark example: Gibbs sampling

Let's see if we can use Cython to speed up MCMC.

Gibbs sampler for function:

$$f(x,y) = x x^2 \exp(-xy^2 - y^2 + 2y - 4x)$$

using conditional distributions:
    
$$x|y \sim Gamma(3, y^2 +4)$$
$$y|x \sim Normal(\frac{1}{1+x}, \frac{1}{2(1+x)})$$

Here is the pure Python implementation:

In [49]:
from numpy import zeros, random, sqrt
gamma = random.gamma
normal = random.normal

def pygibbs(N=20000, thin=200):
    mat = zeros((N,2))
    x,y = mat[0]
    for i in range(N):
        for j in range(thin):
            x = gamma(3, y**2 + 4)
            y = normal(1./(x+1), 1./sqrt(2*(x+1)))
        mat[i] = x,y

    return mat

In [50]:
%timeit pygibbs(1000, 10)

10 loops, best of 3: 22.4 ms per loop


Unchanged, compiling this code with Cython results in a slight improvement in speed.

In [55]:
%%cython

from numpy import zeros, random, sqrt
gamma = random.gamma
normal = random.normal

def cygibbs(N=20000, thin=200):
    mat = zeros((N,2))
    x,y = mat[0]
    for i in range(N):
        for j in range(thin):
            x = gamma(3, y**2 + 4)
            y = normal(1./(x+1), 1./sqrt(2*(x+1)))
        mat[i] = x,y

    return mat

In [57]:
%timeit cygibbs(1000, 10)

10 loops, best of 3: 18.6 ms per loop


Now, for some type declarations:

In [53]:
%%cython

from numpy import zeros, random, sqrt
from numpy cimport *
gamma = random.gamma
normal = random.normal

def cygibbs2(int N=20000, int thin=200):
    cdef: 
        ndarray[float64_t, ndim=2] mat = zeros((N,2))
        float64_t x,y = 0
        int i,j
    for i in range(N):
        for j in range(thin):
            x = gamma(3, y**2 + 4)
            y = normal(1./(x+1), 1./sqrt(2*(x+1)))
        mat[i] = x,y

    return mat

In [54]:
%timeit cygibbs2(1000, 10)

100 loops, best of 3: 15.3 ms per loop


A full-blown "Cythonization" involves using GSL's random number generators, and giving Cython a few more instructions:

In [58]:
# If you are using Homebrew on OS X, you can install GSL using "brew"
!brew install gsl

In [59]:
%%cython -lm -lgsl -lgslcblas

cimport cython
import numpy as np
from numpy cimport *

cdef extern from "math.h":
    double sqrt(double) 
  
cdef extern from "gsl/gsl_rng.h":
    ctypedef struct gsl_rng_type
    ctypedef struct gsl_rng

    gsl_rng_type *gsl_rng_mt19937
    gsl_rng *gsl_rng_alloc(gsl_rng_type * T) nogil
  
cdef extern from "gsl/gsl_randist.h":
    double gamma "gsl_ran_gamma"(gsl_rng * r,double,double)
    double gaussian "gsl_ran_gaussian"(gsl_rng * r,double)
  
cdef gsl_rng *r = gsl_rng_alloc(gsl_rng_mt19937)

@cython.wraparound(False)
@cython.boundscheck(False)
def gibbs(int N=20000,int thin=500):
    cdef: 
        double x=0
        double y=0
        int i, j
        ndarray[float64_t, ndim=2] samples

    samples = np.empty((N,thin))
    for i from 0 <= i < N:
        for j from 0 <= j < thin:
            x = gamma(r,3,1.0/(y*y+4))
            y = gaussian(r,1.0/sqrt(x+1))
        samples[i,0] = x
        samples[i,1] = y
    return samples

In [60]:
%timeit gibbs(1000, 10)

1000 loops, best of 3: 952 µs per loop


## Exercise

Try using Cython to improve the performance of the gradient descent algorithm from our optimization lecture:

In [ ]:
from scipy import optimize

def gradient_descent(x0, f, f_prime, adapt=False):
    x_i, y_i = x0
    all_x_i = list()
    all_y_i = list()
    all_f_i = list()

    for i in range(1, 100):
        all_x_i.append(x_i)
        all_y_i.append(y_i)
        all_f_i.append(f([x_i, y_i]))
        dx_i, dy_i = f_prime(np.asarray([x_i, y_i]))
        if adapt:
            # Compute a step size using a line_search
            step = optimize.line_search(f, f_prime,
                                np.r_[x_i, y_i], -np.r_[dx_i, dy_i],
                                np.r_[dx_i, dy_i], c2=.05)
            step = step[0]
        else:
            step = 1
        x_i += -step*dx_i
        y_i += -step*dy_i
        if np.abs(all_f_i[-1]) < 1e-16:
            break
    return all_x_i, all_y_i, all_f_i

Here is a sample function to optimize. Recall from Section 3 that it returns both the quadratic function and its gradient.

In [ ]:
def quad(epsilon, ndim=2):
    def f(x):
       x = np.asarray(x)
       y = x.copy()
       y *= np.power(epsilon, np.arange(ndim))
       return .33*np.sum(y**2)

    def f_prime(x):
       x = np.asarray(x)
       y = x.copy()
       scaling = np.power(epsilon, np.arange(ndim))
       y *= scaling
       return .33*2*scaling*y

    return f, f_prime

In [ ]:
x0, y0 = 1.6, 1.1

f, f_prime = quad(0.8)
%timeit gd_x_i, gd_y_i, gd_f_i = gradient_descent([x0, y0], f, f_prime)

In [ ]:
# Write answer here

## Numba

Cython precompiles parts of Python code before running. Another approach is **Just-in-Time (JIT)** compilation. Numba is a compiler that runs Python code through an LLVM compiler to produce optimized bytecode for fast execution. Numba does not require a C/C++ compiler on your machine.

Numba's lone API is a **decorator**.

The `@jit` decorator runs the decorated function through bytecode analysis and the function arguments through a type inference engine, and generates an intermediate representation of your code, which is then passed to LLVM for compilation to bytecode.

In [ ]:
!pip install numba

In [61]:
from numba import jit

In [62]:
@jit
def nfibonacci(size):
    F = np.empty(size, 'int')
    a, b = 0, 1
    for i in range(size):
        F[i] = a
        a, b = b, a + b
    return F

In [63]:
nfibonacci(50)

array([         0,          1,          1,          2,          3,
                5,          8,         13,         21,         34,
               55,         89,        144,        233,        377,
              610,        987,       1597,       2584,       4181,
             6765,      10946,      17711,      28657,      46368,
            75025,     121393,     196418,     317811,     514229,
           832040,    1346269,    2178309,    3524578,    5702887,
          9227465,   14930352,   24157817,   39088169,   63245986,
        102334155,  165580141,  267914296,  433494437,  701408733,
       1134903170, 1836311903, 2971215073, 4807526976, 7778742049])

Numba is able to compile separate specializations depending on the input types.

If you want fine-grained control over types chosen by the compiler, you can tell Numba the function signature (types) to expect.

In [64]:
from numba import int32

@jit(int32[:](int32))
def nfibonacci(size):
    F = np.empty(size, 'int')
    a, b = 0, 1
    for i in range(size):
        F[i] = a
        a, b = b, a + b
    return F

In [65]:
nfibonacci(50)

array([         0,          1,          1,          2,          3,
                5,          8,         13,         21,         34,
               55,         89,        144,        233,        377,
              610,        987,       1597,       2584,       4181,
             6765,      10946,      17711,      28657,      46368,
            75025,     121393,     196418,     317811,     514229,
           832040,    1346269,    2178309,    3524578,    5702887,
          9227465,   14930352,   24157817,   39088169,   63245986,
        102334155,  165580141,  267914296,  433494437,  701408733,
       1134903170, 1836311903, 2971215073, 4807526976, 7778742049])

Compilation is deferred until the first function execution. Numba will infer the argument types at call time, and generate optimized code based on this information. 

In [66]:
def pairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M), dtype=np.float)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

X = np.random.random((1000, 3))

%timeit pairwise_python(X)

1 loops, best of 3: 3.85 s per loop


In [67]:
@jit
def npairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M), dtype=np.float)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

In [68]:
%timeit npairwise_python(X)

The slowest run took 35.03 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 9.38 ms per loop


Numba-compiled functions can call other compiled functions. In some situations, the optimizer may even inline the function in the machine code code.

In [69]:
def square(x):
    return x ** 2

def hypot(x, y):
    return np.sqrt(square(x) + square(y))

In [70]:
%timeit hypot(10, 8)

The slowest run took 11.07 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 2.56 µs per loop


In [71]:
@jit
def nsquare(x):
    return x ** 2

@jit
def nhypot(x, y):
    return np.sqrt(nsquare(x) + nsquare(y))

In [72]:
%timeit nhypot(10, 8)

The slowest run took 637214.93 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 279 ns per loop


Numba can compile *most* NumPy functions, as well as generators.

Numba does *not* compile things like lists, sets, dictionaries (tuples are compiled), comprehensions, and string operations, so there will be no speedup for these.

The best strategy is not to apply the `@jit` decorator all over your code, but to use Python's profiling tools to identify "hotspots" in your program, and selectively apply `@jit`.

## References

van der Plas, J. (2014) [Why Python is Slow: Looking Under the Hood](https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/)

van der Plas, J. (2015) [Optimizing Python in the Real World: NumPy, Numba, and the NUFFT](https://jakevdp.github.io/blog/2015/02/24/optimizing-python-with-numpy-and-numba/)

[A guide to analyzing Python performance](http://www.huyng.com/posts/python-performance-analysis/)

[Kurt Smith's Cython tutorial from SciPy 2013](https://www.youtube.com/watch?v=JKCjsRDffXo)
